In [0]:
# 네이버 웹툰 댓글을 가져와보세요.

In [0]:
import requests
from bs4 import BeautifulSoup

In [9]:
comment_url = 'https://comic.naver.com/comment/comment.nhn?titleId=183559&no=485'

resp = requests.get(comment_url)
resp

<Response [200]>

In [10]:
soup = BeautifulSoup(resp.content, 'lxml')
div_tag = soup.select('div.u_cbox_content_wrap')
div_tag

[]

BeautifulSoup Select로 파싱이 안된다.  

JS인지 확인한다.  

JS라면, Network 탭에서 JS 클릭하고, 창에서 신호를 줘서 나오는 대상의 Preview를 확인한다.  

Preview에 원하는 정보가 들어가 있는지 확인한다.  
  
원하는 정보가 들어가 있다면, Headers의 Requests URL로 다시 처음부터 시작한다.

In [11]:
comment_url = 'https://apis.naver.com/commentBox/cbox/web_naver_list_jsonp.json?ticket=comic&templateId=webtoon&pool=cbox3&_callback=jQuery112407830826950284522_1591579730624&lang=ko&country=KR&objectId=183559_485&categoryId=&pageSize=15&indexSize=10&groupId=&listType=OBJECT&pageType=default&page=1&refresh=true&sort=best&cleanbotGrade=2&_=1591579730627'

resp = requests.get(comment_url)
resp

<Response [200]>

preview에서 확인했듯, 데이터 형식이 JSON이었다.  
JSON은 select와 같은 파싱 함수를 사용할 수 없다.  
사용할 필요가 없다.  
쉽게, 딕셔너리의 키값으로 밸류값을 지칭하듯,  
키값으로 파싱하면 된다.

In [12]:
#json이기 때문에, 눈으로 보면서 키값을 찾아야하므로, resp.text를 사용한다.
resp.text

'jQuery112407830826950284522_1591579730624({"success":false,"code":"3999","message":"잘못된 접근입니다.","lang":"ko","country":"KR","result":{},"date":"2020-06-08T03:01:21+0000"});'

내가 직접 브라우저로 접근했을 때는 성공이라고 리턴했으나,  
크롤러로 접근하니 잘못된 접근이라고 리턴이 된다면,  
  
headers를 바꿔줘야한다.  
== 크롤러가 사람인거처럼 보이도록 바꿔줘야 한다.  

크롤러와 사람의 차이는 headers다!

In [13]:
comment_url = 'https://apis.naver.com/commentBox/cbox/web_naver_list_jsonp.json?ticket=comic&templateId=webtoon&pool=cbox3&_callback=jQuery112407830826950284522_1591579730624&lang=ko&country=KR&objectId=183559_485&categoryId=&pageSize=5000&indexSize=10&groupId=&listType=OBJECT&pageType=default&page=1&refresh=true&sort=best&cleanbotGrade=2&_=1591579730627'

header = {'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/81.0.4044.138 Safari/537.36',
          'referer': 'https://comic.naver.com/comment/comment.nhn?titleId=183559&no=485'}

resp = requests.get(comment_url, headers=header)
resp

<Response [200]>

In [14]:
resp.text

'jQuery112407830826950284522_1591579730624({"success":true,"code":"1000","message":"요청을 성공적으로 처리하였습니다.","lang":"ko","country":"KR","result":{"commentList":[{"ticket":"comic","objectId":"183559_485","categoryId":"*","templateId":"default","commentNo":389787631,"parentCommentNo":389787631,"replyLevel":1,"replyCount":0,"replyAllCount":0,"replyPreviewNo":null,"replyList":null,"imageCount":0,"imageList":null,"imagePathList":null,"imageWidthList":null,"imageHeightList":null,"commentType":"txt","stickerId":null,"sticker":null,"sortValue":1589725478064,"contents":"스나이퍼 재평가..  스나이퍼 업적 : 엘파시온 1킬, 지파장 1킬 도합2킬 + 정보제공 \\n\\n이정도면 스파이 아니냐ㅋㅋ","userIdNo":"azRVL","exposedUserIp":null,"lang":"ko","country":"KR","idType":"naver","idProvider":"naver","userName":"Jang","userProfileImage":"https://profile-phinf.pstatic.net/28683/dd0b9ca85ea0cd8d2356202173f13555ef004ea34896c91ef510f7bdbde3f723.jpg","profileType":"naver","modTime":"2020-05-17T23:24:38+0900","modTimeGmt":"2020-05-17T14:24:38+0000","regTime":"20

In [15]:
import re

resp_json = re.findall('\((.+)\)', resp.text)[0] # [0]해주는 이유는 findall은 리스트로 리턴하기 때문에
resp_json

'{"success":true,"code":"1000","message":"요청을 성공적으로 처리하였습니다.","lang":"ko","country":"KR","result":{"commentList":[{"ticket":"comic","objectId":"183559_485","categoryId":"*","templateId":"default","commentNo":389787631,"parentCommentNo":389787631,"replyLevel":1,"replyCount":0,"replyAllCount":0,"replyPreviewNo":null,"replyList":null,"imageCount":0,"imageList":null,"imagePathList":null,"imageWidthList":null,"imageHeightList":null,"commentType":"txt","stickerId":null,"sticker":null,"sortValue":1589725478064,"contents":"스나이퍼 재평가..  스나이퍼 업적 : 엘파시온 1킬, 지파장 1킬 도합2킬 + 정보제공 \\n\\n이정도면 스파이 아니냐ㅋㅋ","userIdNo":"azRVL","exposedUserIp":null,"lang":"ko","country":"KR","idType":"naver","idProvider":"naver","userName":"Jang","userProfileImage":"https://profile-phinf.pstatic.net/28683/dd0b9ca85ea0cd8d2356202173f13555ef004ea34896c91ef510f7bdbde3f723.jpg","profileType":"naver","modTime":"2020-05-17T23:24:38+0900","modTimeGmt":"2020-05-17T14:24:38+0000","regTime":"2020-05-17T23:24:38+0900","regTimeGmt":"2020

문자열(JSON) -> 딕셔너리

In [16]:
import json

resp_dic = json.loads(resp_json)
for idx, commentList in enumerate(resp_dic['result']['commentList']):
  print(idx, commentList['contents'])

0 스나이퍼 재평가..  스나이퍼 업적 : 엘파시온 1킬, 지파장 1킬 도합2킬 + 정보제공 

이정도면 스파이 아니냐ㅋㅋ
1 헐 저거 구스구스아님? 20층에서 시험본곳 총시험감독관!

청노스승 뮬러브도 나오는건가? 솔직히 밤스승중
러브가 제일쓸모있는기술 알려준 참스승이라 생각한다 나는

재등장가나요!?
2 밤이 여태까지 살아 남게 해준 장본인 러브.. 참스승 께서 밤을 위해 오셧다 마지막 보고 소름돋았네
3 막컷 진짜 소름이다... 이거 예상한 사람 단언컨대 단 한 명도 없었을듯
4 자꾸 질질끈다 이래서 작가님이 하천희 유한성 싸움씬스킵하셨네..
5 부모님이 fug때문에 죽었는데 도와주러 온거임!?
역시 참스승 러브 이때까지 밤이 살아있는 이유는 청노 때문지..
6 밤스승목록
퀀트:역흐름제어
러브:청노
유한성:신수제어술
하진성:화접공파술
데이터에드안:오브
에반켈:오브
밤 스승의날때 고생좀햇을듯

7 와...레파브브 하드캐리 미뗬네
1.자하드군 부군단장 강제 희생시킴
2.에반켈 압축해체
3.로 포 비아 지파장&amp;가문을 상징하는 가주의 신수 사살
4.3번으로 인해 화이트 부활
5.내부고발자
첫번째 방벽의 전투는 누가뭐래도 레파브브가 1등공신이다
8 둥지 자체가 하나의 어항인듯 합니다.
스나이퍼가 있던 더미속에서 신해어가 나왔듯이
둥지도 하나의 어항이여서 거대한 신해어 혹은 로포비아 가주가 직접 기른 무언가가 등장할것 같음
9 스나이퍼 누나 이쁜데 이유가 잇엇네
10 마지막에 나온두명은 뮬러브(2부6화첫등장,밤이청노카피한스승)
아우구스구스(2부22화등장,20층 총시험관,fug편)입니다.
11 구스구스 : 후라이드로 넘어와
뮬러브 : 싫습니다
구스구스 : 너 나 싫지?
뮬러브 : 싫습니다
12 마지막짤 구스구스 옆에 러브가 떡 하니 그려져 있습니다!!!!!!!
20층에서 마지막으로 보고 지금 엄청 강해진 밤의 모습을 볼 참스승 러브의 반응이 궁금하네요
13 솔직히 밤측세력 다모인거 아니지않나? 생각날만한 조력자만해도 어마어마한데... 야스라챠나 군단장들이 밤 

In [17]:
pages = resp_dic['result']['count']['comment'] // 15 + 1
pages

387

In [18]:
from tqdm.notebook import tqdm

commentLists = []
for page in tqdm(range(pages)):
    comment_url = 'https://apis.naver.com/commentBox/cbox/web_naver_list_jsonp.json'

    params = {
        'ticket':'comic',
        'templateId':'webtoon', 
        'pool':'cbox3',
        '_callback':'jQuery112407830826950284522_1591579730624',
        'lang':'ko',
        'country':'KR',
        'objectId':'183559_485', # objectId = 웹툰ID_화정보
        'pageSize':15,
        'indexSize':10,
        'listType':'OBJECT',
        'page': page+1,
        'cleanbotGrade':2,
        'sort':'new'
        }

    header = {'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/81.0.4044.138 Safari/537.36',
              'referer': 'https://comic.naver.com/comment/comment.nhn?titleId=183559&no=485'}

    resp = requests.get(comment_url, params, headers=header)

    resp_json = re.findall('\((.+)\)', resp.text)[0] # [0]해주는 이유는 findall은 리스트로 리턴하기 때문에

    for commentList in resp_dic['result']['commentList']:
        commentLists.append(commentList['contents'])

In [19]:
commentLists

['스나이퍼 재평가..  스나이퍼 업적 : 엘파시온 1킬, 지파장 1킬 도합2킬 + 정보제공 \n\n이정도면 스파이 아니냐ㅋㅋ',
 '헐 저거 구스구스아님? 20층에서 시험본곳 총시험감독관!\n\n청노스승 뮬러브도 나오는건가? 솔직히 밤스승중\n러브가 제일쓸모있는기술 알려준 참스승이라 생각한다 나는\n\n재등장가나요!?',
 '밤이 여태까지 살아 남게 해준 장본인 러브.. 참스승 께서 밤을 위해 오셧다 마지막 보고 소름돋았네',
 '막컷 진짜 소름이다... 이거 예상한 사람 단언컨대 단 한 명도 없었을듯',
 '자꾸 질질끈다 이래서 작가님이 하천희 유한성 싸움씬스킵하셨네..',
 '부모님이 fug때문에 죽었는데 도와주러 온거임!?\n역시 참스승 러브 이때까지 밤이 살아있는 이유는 청노 때문지..',
 '밤스승목록\n퀀트:역흐름제어\n러브:청노\n유한성:신수제어술\n하진성:화접공파술\n데이터에드안:오브\n에반켈:오브\n밤 스승의날때 고생좀햇을듯\n',
 '와...레파브브 하드캐리 미뗬네\n1.자하드군 부군단장 강제 희생시킴\n2.에반켈 압축해체\n3.로 포 비아 지파장&amp;가문을 상징하는 가주의 신수 사살\n4.3번으로 인해 화이트 부활\n5.내부고발자\n첫번째 방벽의 전투는 누가뭐래도 레파브브가 1등공신이다',
 '둥지 자체가 하나의 어항인듯 합니다.\n스나이퍼가 있던 더미속에서 신해어가 나왔듯이\n둥지도 하나의 어항이여서 거대한 신해어 혹은 로포비아 가주가 직접 기른 무언가가 등장할것 같음',
 '스나이퍼 누나 이쁜데 이유가 잇엇네',
 '마지막에 나온두명은 뮬러브(2부6화첫등장,밤이청노카피한스승)\n아우구스구스(2부22화등장,20층 총시험관,fug편)입니다.',
 '구스구스 : 후라이드로 넘어와\n뮬러브 : 싫습니다\n구스구스 : 너 나 싫지?\n뮬러브 : 싫습니다',
 '마지막짤 구스구스 옆에 러브가 떡 하니 그려져 있습니다!!!!!!!\n20층에서 마지막으로 보고 지금 엄청 강해진 밤의 모습을 볼 참스승 러브의 반응이 궁금하네요',
 '솔직히 밤측세